In [1]:
!pip install scapy

In [2]:
import os
import requests
import gzip
import shutil

# URL do arquivo a ser baixado
url = "http://mawi.wide.ad.jp/mawi/samplepoint-F/2007/200701251400.dump.gz"
# Nome do arquivo compactado (.gz)
gz_filename = "200701251400.dump.gz"
# Nome do arquivo de saída após descompactação
pcap_filename = "200701251400.pcap"

# Baixa o arquivo gzip somente se ainda não existir localmente
if not os.path.exists(gz_filename):
    print("Baixando arquivo...")
    response = requests.get(url)
    with open(gz_filename, "wb") as f:
        f.write(response.content)
    print("Download concluído.")

# Descompacta o arquivo gzip somente se o arquivo .pcap ainda não existir
if not os.path.exists(pcap_filename):
    print("Descompactando arquivo...")
    with gzip.open(gz_filename, "rb") as f_in:
        with open(pcap_filename, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)
    print(f"Arquivo descompactado como: {pcap_filename}")
else:
    print("Arquivo já está descompactado.")


Arquivo já está descompactado.


In [3]:
from scapy.all import PcapReader

# Lê os primeiros 5 pacotes sem carregar todo o arquivo
with PcapReader("200701251400.pcap") as pcap_reader:
    for i, pkt in enumerate(pcap_reader):
        print(f"Pacote {i+1}")
        pkt.show()
        # Aqui testamos apenas os 3 primeiros pacotes e interrompemos a leitura
        if i >= 2:
            break


Pacote 1
###[ Ethernet ]###
  dst       = 00:0e:39:e3:34:00
  src       = 00:90:69:ec:ad:5c
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 52
     id        = 12534
     flags     = DF
     frag      = 0
     ttl       = 52
     proto     = tcp
     chksum    = 0xcd05
     src       = 43.144.42.46
     dst       = 205.249.37.17
     \options   \
###[ TCP ]###
        sport     = 58987
        dport     = 51935
        seq       = 3183539652
        ack       = 4234212667
        dataofs   = 8
        reserved  = 0
        flags     = A
        window    = 34032
        chksum    = 0xc652
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (1307315593, 758645654))]

Pacote 2
###[ Ethernet ]###
  dst       = 00:90:69:ec:ad:5c
  src       = 00:0e:39:e3:34:00
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 40
     id        = 2915
     

In [4]:
from scapy.all import PcapReader, IP, TCP, UDP
import pandas as pd

rows = []
N = 9999999
# Abrir o arquivo pcap e iterar pelos pacotes
with PcapReader("200701251400.pcap") as pcap_reader:
    for i, pkt in enumerate(pcap_reader):
        if IP in pkt:
            try:
                timestamp = float(pkt.time) if hasattr(pkt, 'time') else None
            except Exception:
                timestamp = None

            # Criar linha com as chaves em inglês para o DataFrame
            row = {
                "Packet": i + 1,
                "Timestamp": timestamp,
                "Source IP": pkt[IP].src,
                "Destination IP": pkt[IP].dst,
                "Protocol": pkt[IP].proto,
                "Size (bytes)": len(pkt)
            }

            # Preencher campos de porta e tipo de protocolo
            if TCP in pkt:
                row["Source Port"] = pkt[TCP].sport
                row["Destination Port"] = pkt[TCP].dport
                row["Type"] = "TCP"
            elif UDP in pkt:
                row["Source Port"] = pkt[UDP].sport
                row["Destination Port"] = pkt[UDP].dport
                row["Type"] = "UDP"
            else:
                row["Source Port"] = None
                row["Destination Port"] = None
                row["Type"] = "Outro"

            rows.append(row)

        # Limitamos o número de pacotes processados (ajustável conforme a RAM)
        if i >= N:
            break

# Criar um DataFrame com as linhas coletadas
df_packets = pd.DataFrame(rows)

# Filtrar timestamps válidos e converter para datetime
df_packets = df_packets[pd.to_numeric(df_packets['Timestamp'], errors='coerce').notnull()]
df_packets['Timestamp'] = pd.to_datetime(df_packets['Timestamp'].astype(float), unit='s')

# Exibir os primeiros registros
df_packets.head()


,Packet,Timestamp,Source IP,Destination IP,Protocol,Size (bytes),Source Port,Destination Port,Type
0,1,2007-01-25 05:00:00.718190908,43.144.42.46,205.249.37.17,6,66,58987.0,51935.0,TCP
1,2,2007-01-25 05:00:00.718199015,158.173.82.53,81.65.180.183,6,54,1159.0,80.0,TCP
2,3,2007-01-25 05:00:00.718310118,204.37.161.167,208.80.39.232,6,54,63046.0,80.0,TCP
3,4,2007-01-25 05:00:00.718317032,219.224.16.164,167.41.75.31,6,54,80.0,1324.0,TCP
4,5,2007-01-25 05:00:00.718322039,158.173.176.187,104.243.187.6,6,66,33796.0,80.0,TCP


In [5]:
# Agrupar o tráfego por segundo (usa "Size (bytes)" conforme definido acima)
df_traffic = df_packets.copy()
df_traffic['second'] = df_traffic['Timestamp'].dt.floor('s')
traffic_per_sec = df_traffic.groupby('second')['Size (bytes)'].sum().reset_index()
traffic_per_sec.rename(columns={'Size (bytes)': 'bytes'}, inplace=True)
traffic_per_sec.head()


,second,bytes
0,2007-01-25 05:00:00,418102
1,2007-01-25 05:00:01,1334248
2,2007-01-25 05:00:02,1349866
3,2007-01-25 05:00:03,1416918
4,2007-01-25 05:00:04,1446890


In [6]:
df_packets

,Packet,Timestamp,Source IP,Destination IP,Protocol,Size (bytes),Source Port,Destination Port,Type
0,1,2007-01-25 05:00:00.718190908,43.144.42.46,205.249.37.17,6,66,58987.0,51935.0,TCP
1,2,2007-01-25 05:00:00.718199015,158.173.82.53,81.65.180.183,6,54,1159.0,80.0,TCP
2,3,2007-01-25 05:00:00.718310118,204.37.161.167,208.80.39.232,6,54,63046.0,80.0,TCP
3,4,2007-01-25 05:00:00.718317032,219.224.16.164,167.41.75.31,6,54,80.0,1324.0,TCP
4,5,2007-01-25 05:00:00.718322039,158.173.176.187,104.243.187.6,6,66,33796.0,80.0,TCP
...,...,...,...,...,...,...,...,...,...
9972804,9999996,2007-01-25 05:07:20.097739935,40.0.207.98,204.37.49.208,6,54,80.0,2961.0,TCP
9972805,9999997,2007-01-25 05:07:20.097852945,40.0.207.98,204.37.49.208,6,54,80.0,2961.0,TCP
9972806,9999998,2007-01-25 05:07:20.097978115,40.0.207.98,204.37.49.208,6,54,80.0,2961.0,TCP
9972807,9999999,2007-01-25 05:07:20.097984076,205.142.80.86,205.249.32.44,6,54,24064.0,12426.0,TCP


In [7]:
# Copiar DataFrame original
df = df_packets.copy()

# Garantir que 'Timestamp' seja datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])  # aqui garantimos o tipo datetime

# Arredondar para o segundo
df['second'] = df['Timestamp'].dt.floor('s')  # arredondando para o segundo mais próximo

# Agrupar o tráfego por segundo usando a chave 'Size (bytes)' definida acima
traffic_per_sec = df.groupby('second')['Size (bytes)'].sum().reset_index()

# Renomear coluna para 'bytes' conforme convenção
traffic_per_sec.rename(columns={'Size (bytes)': 'bytes'}, inplace=True)


In [8]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Parâmetro configurável: comprimento da janela de tempo (número de instantes usados para prever o próximo)
seq_len = 10  # ajuste este valor conforme a granularidade temporal e a capacidade de memória 'janelas'
# Extrair os valores de tráfego (coluna 'bytes') como array 2D para o scaler
data_values = traffic_per_sec[['bytes']].values  # shape: (n_amostras, 1)
# Normalização Min-Max para [0, 1]:
# - Evita que valores grandes dominem o treino do LSTM
# - Facilita a convergência do otimizador
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_values)

In [9]:
def create_sequences(data, seq_len):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    return np.array(X), np.array(y)

In [10]:
# Criar as sequências de treino
# Ajusta seq_len para no máximo (n_amostras – 1)
max_seq = len(data_scaled) - 1
if max_seq <= 0:
    raise ValueError(
        f"Não há dados suficientes ({len(data_scaled)} pontos) para criar sequer uma sequência."
    )
seq_len = min(seq_len, max_seq)
X, y = create_sequences(data_scaled, seq_len=seq_len)
print(f"Usando seq_len = {seq_len}, X.shape = {X.shape}")

Usando seq_len = 10, X.shape = (431, 10, 1)


In [11]:
print("→ traffic_per_sec:", len(traffic_per_sec), "filas")
print("→ data_scaled.shape:", data_scaled.shape)
print("→ X.ndim, X.shape:", X.ndim, X.shape)
print("→ y.ndim, y.shape:", y.ndim, y.shape)


→ traffic_per_sec: 441 filas
→ data_scaled.shape: (441, 1)
→ X.ndim, X.shape: 3 (431, 10, 1)
→ y.ndim, y.shape: 2 (431, 1)


In [12]:
# Definir tamanho do conjunto de treino (80%) e validação (20%)
train_size = int(len(X) * 0.8)
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]

# Aqui testamos quantas amostras ficaram em cada subconjunto
print(f"Tamanho de X_train: {X_train.shape}")  # número de sequências de treino
print(f"Tamanho de X_val:   {X_val.shape}")    # número de sequências de validação
print(f"Tamanho de y_train: {y_train.shape}")  # rótulos de treino
print(f"Tamanho de y_val:   {y_val.shape}")    # rótulos de validação


Tamanho de X_train: (344, 10, 1)
Tamanho de X_val:   (87, 10, 1)
Tamanho de y_train: (344, 1)
Tamanho de y_val:   (87, 1)
